In [1]:
import sits.SITS as sits
import folium
from datetime import datetime
import time

In [2]:
import geopandas as gpd
import pandas as pd

in_vect = "france_barkbeetle_lite_single.geojson"
test_2154 = sits.Vec2gdf(in_vect)
test_2154.gdf.crs.to_epsg()

2154

In [3]:
# calculates the boundiug box for each feature.
test_2154.set_bbox('gdf')
test_2154.bbox.crs.to_epsg()

2154

In [4]:
#f = folium.Figure(height=400)
#m = folium.Map(location=[55, 0], zoom_start=3).add_to(f)
#test_2154.gdf.explore(m=m, height=400, color='green')
#test_2154.bbox.explore(m=m, height=400)

In [5]:
type(test_2154.bbox)

geopandas.geodataframe.GeoDataFrame

In [6]:
test_process = test_2154.bbox.to_crs(4326)

test_process = pd.concat([test_process, test_process['geometry'].bounds], axis=1)
test_process['bbox_4326'] = test_process[['minx', 'miny', 'maxx', 'maxy']].values.tolist()


In [7]:
test_3035 = test_2154.bbox.to_crs(3035)
test_3035_bounds = pd.concat([test_3035, test_3035['geometry'].bounds], axis=1)
test_3035_bounds['bbox_3035'] = test_3035_bounds[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

test_process = pd.concat([test_process, test_3035_bounds['bbox_3035']], axis=1)
test_process['bbox_tuple'] = test_process.apply(lambda row: (row['bbox_4326'], row['bbox_3035']), axis=1)
test_process['bbox_tuple'][0]


([2.8255044256228508,
  45.634941277411386,
  2.8260189959120745,
  45.6353022635073],
 [3761787.7849638816,
  2529437.8009796273,
  3761831.262662839,
  2529481.4105150606])

In [8]:
#%%time

#imgcoll = sits.StacAttack()
#imgcoll.searchItems(total_bbox_4326, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))

In [9]:
#len(imgcoll.items)

In [10]:
from dask import delayed, compute

In [11]:
def f_test(aoi_bounds_latlong, aoi_bounds_proj, gid, rep):
    # loads time-series images in EPSG:3035
    imgcoll = sits.StacAttack()
    imgcoll.searchItems(aoi_bounds_latlong, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))
    imgcoll.loadImgs(aoi_bounds_proj, crs_out=3035)
    # exports time-series into csv file and netCDF file
    imgcoll.to_nc(f"/eos/jeodpp/home/users/osekenj/data/temp_data/{rep}", gid, 'image')


In [12]:
for i in test_process['bbox_tuple'][:20]:
    print (i[0],i[1])

[2.8255044256228508, 45.634941277411386, 2.8260189959120745, 45.6353022635073] [3761787.7849638816, 2529437.8009796273, 3761831.262662839, 2529481.4105150606]
[6.139293359502604, 45.587155051477055, 6.139693095432569, 45.587435718717316] [4019438.9740521642, 2504995.883254013, 4019471.590462089, 2505028.4940533843]
[5.534088480344151, 48.46721314992736, 5.534491045898747, 48.46777132050433] [3990846.743521249, 2827079.0831763, 3990880.024332588, 2827142.598343773]
[5.534190109814471, 48.46654256305175, 5.534504293061137, 48.467286789130924] [3990849.826223296, 2827004.636245847, 3990877.832390555, 2827088.324802132]
[5.540081457732796, 48.46730642585579, 5.541369223500214, 48.46819730023053] [3991289.9031796386, 2827058.74595494, 3991390.4657415943, 2827163.005502219]
[5.53690687375325, 48.463269707304995, 5.537363556439186, 48.46337336273527] [3991028.9235258377, 2826628.4510618323, 3991063.193960321, 2826641.9578250065]
[5.190300960857002, 48.690498561289914, 5.190777139236365, 48.69

In [13]:
%%time
#test 01

"""
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    f_test(i[0], i[1], gid, "test01")
"""

CPU times: user 12 µs, sys: 4 µs, total: 16 µs
Wall time: 31.7 µs


'\nfor gid, i in enumerate(test_process[\'bbox_tuple\'][:20]):\n    f_test(i[0], i[1], gid, "test01")\n'

In [14]:
%%time
#test 02
"""
fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test02")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask)#, scheduler='processes')
"""

CPU times: user 0 ns, sys: 15 µs, total: 15 µs
Wall time: 30.5 µs


'\nfetch_dask = []\n    \nfor gid, i in enumerate(test_process[\'bbox_tuple\'][:20]):\n    single = delayed(f_test)(i[0], i[1], gid, "test02")\n    fetch_dask.append(single)\n\nresults_dask = compute(*fetch_dask)#, scheduler=\'processes\')\n'

In [15]:
%%time
#test 03
"""
fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test03")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask, scheduler='processes')
"""
# CPU times: user 74.2 ms, sys: 14.8 ms, total: 89 ms
# Wall time: 4min 8s

CPU times: user 13 µs, sys: 4 µs, total: 17 µs
Wall time: 34.1 µs


'\nfetch_dask = []\n    \nfor gid, i in enumerate(test_process[\'bbox_tuple\'][:20]):\n    single = delayed(f_test)(i[0], i[1], gid, "test03")\n    fetch_dask.append(single)\n\nresults_dask = compute(*fetch_dask, scheduler=\'processes\')\n'

In [16]:
#outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test04b"
#test = sits.Multiproc2('image', 'nc', outdir)

In [17]:
#for gid, i in enumerate(test_process['bbox_tuple'][:5]):
#    print(i[0], i[1], gid)
#    test.fetch_func(i[0], i[1], gid, resolution=20)

In [18]:
#test.fetch_dask

In [19]:
#%%time
#test.dask_compute()

In [20]:
%%time
outdir = "/eos/jeodpp/home/users/osekenj/data/temp_data/test05"
test = sits.Multiproc2('patch', 'nc', outdir)
for gid, i in enumerate(test_process['bbox_tuple'][:5]):
    print(i[0], i[1], gid)
    test.fetch_func(i[0], i[1], gid, dimx=10, dimy=10)
test.dask_compute()

[2.8255044256228508, 45.634941277411386, 2.8260189959120745, 45.6353022635073] [3761787.7849638816, 2529437.8009796273, 3761831.262662839, 2529481.4105150606] 0
[6.139293359502604, 45.587155051477055, 6.139693095432569, 45.587435718717316] [4019438.9740521642, 2504995.883254013, 4019471.590462089, 2505028.4940533843] 1
[5.534088480344151, 48.46721314992736, 5.534491045898747, 48.46777132050433] [3990846.743521249, 2827079.0831763, 3990880.024332588, 2827142.598343773] 2
[5.534190109814471, 48.46654256305175, 5.534504293061137, 48.467286789130924] [3990849.826223296, 2827004.636245847, 3990877.832390555, 2827088.324802132] 3
[5.540081457732796, 48.46730642585579, 5.541369223500214, 48.46819730023053] [3991289.9031796386, 2827058.74595494, 3991390.4657415943, 2827163.005502219] 4


/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


10 10
3 6
4 4
3 3
3 8


PermissionError: [Errno 13] Permission denied: '/eos/jeodpp/home/users/osekenj/data/temp_data/test05/S2_fid-1_patch_2023-01-01 00:00:00-2023-12-31 00:00:00.nc'

Traceback
---------
  File "/home/osekenj/.local/lib/python3.11/site-packages/dask/local.py", line 225, in execute_task
    result = _execute_task(task, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/osekenj/.local/lib/python3.11/site-packages/dask/core.py", line 127, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/osekenj/.local/lib/python3.11/site-packages/dask/utils.py", line 78, in apply
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/eos/jeodpp/home/users/osekenj/data/github_proj/SITS_dev/SITS_utils/sits/SITS.py", line 604, in fdask
    imgcoll.to_nc(self.outdir, gid, self.arrtype)
  File "/eos/jeodpp/home/users/osekenj/data/github_proj/SITS_dev/SITS_utils/sits/SITS.py", line 471, in to_